In [4]:
import pandas as pd
import numpy as np
import pickle
import xgboost as xgb
import datetime
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, classification_report
pd.set_option('display.max_columns', None)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [5]:
df=pd.read_csv('DATA.csv')

In [8]:
name=df['NAME'].unique()

In [9]:
name

array(['Mahendranagar', 'Dhangadhi', 'Nepalgunj', 'Krishnanagar',
       'Bahadurganj', 'Taulihawa', 'Butwal', 'Bhairahawa', 'Hemja',
       'Pokhara', 'Bharatpur', 'Birgunj', 'Kalaiya', 'Hetauda',
       'Janakpur', 'KTM', 'Gaur', 'Banepa', 'Malangwa', 'Gaushala',
       'Jaleshwor', 'Lahan', 'Rajbiraj', 'Dharan', 'Biratnagar',
       'Bhadrapur'], dtype=object)

In [10]:
ratio=pd.DataFrame([0.25, 43, 0, 10,
                    0, 3.5, 30.4, 2.2, 44.5,
                    58, 4.25, 0, 0, 19.5,
                    19.3, 69.5, 0.33, 124.6, 38, 0,
                    0, 12.6, 1.67, 21.8, 3, 0.83], 
                    index=name, columns=['source_ecoli'])

In [11]:
ratio=ratio.reset_index()
ratio=ratio.rename(columns={'index':'NAME'})

In [12]:
DF=pd.merge(df, ratio, on='NAME', how='inner')

In [16]:
DF.to_csv('DATA.csv')

In [15]:
DF.isna().sum()

Unnamed: 0             0
NAME                   0
Rowid                  0
XY水道水位置情報              0
X                      0
Y                      0
up_down                0
curvature              0
inclination            0
tilt_direction         0
altitude               0
disto_river            2
disto_stations         0
disto_mainroad         0
disto_syorizyo         0
supply_hours           0
no_water_days          0
total_population       0
population_served      0
popu-served            0
number_taps            0
pipelength             0
pipelength_per_pipe    0
served/pipes           0
(popu-served)/pipes    0
oldest_pipe_age        0
ST                     0
RSF                    0
FL                     0
PF                     0
RF                     0
EColi.                 0
ratio                  0
source_ecoli           0
dtype: int64

In [17]:
df=DF

In [18]:
df=df.dropna(axis=0)

In [19]:
df.columns

Index(['Unnamed: 0', 'NAME', 'Rowid', 'XY水道水位置情報', 'X', 'Y', 'up_down',
       'curvature', 'inclination', 'tilt_direction', 'altitude', 'disto_river',
       'disto_stations', 'disto_mainroad', 'disto_syorizyo', 'supply_hours',
       'no_water_days', 'total_population ', 'population_served',
       'popu-served', 'number_taps', 'pipelength', 'pipelength_per_pipe',
       'served/pipes', '(popu-served)/pipes', 'oldest_pipe_age', 'ST', 'RSF',
       'FL', 'PF', 'RF', 'EColi.', 'ratio', 'source_ecoli'],
      dtype='object')

In [20]:
y=df['EColi.']
X=df.loc[:, ['up_down', 'curvature', 'inclination', 'tilt_direction', 'altitude', 'disto_river', 'disto_stations', 
             'disto_mainroad', 'disto_syorizyo', 'supply_hours', 'no_water_days', 'total_population ', 'population_served',
             'popu-served', 'number_taps', 'pipelength', 'pipelength_per_pipe', 'served/pipes', '(popu-served)/pipes', 
             'oldest_pipe_age', 'ST', 'RSF', 'FL', 'PF', 'RF',  'ratio', 'source_ecoli']]

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3,
                                                  random_state=1, stratify=y)
print('Labels counts in y:', np.bincount(y))
print('Lables counts in y_train:', np.bincount(y_train))
print('Lables counts in y_test:', np.bincount(y_test))

Labels counts in y: [ 98 120]
Lables counts in y_train: [68 84]
Lables counts in y_test: [30 36]


In [22]:
X_train_std=(X_train-X_train.mean(axis=0))/X_train.std(axis=0)
X_test_std=(X_test-X_train.mean(axis=0))/X_train.std(axis=0)

In [23]:
def threshold(fare):
    if fare<=0:
        return 0
    else:
        return 1

In [24]:
X_train_std['ST']=X_train_std['ST'].apply(threshold)
X_train_std['RSF']=X_train_std['RSF'].apply(threshold)
X_train_std['FL']=X_train_std['FL'].apply(threshold)
X_train_std['PF']=X_train_std['PF'].apply(threshold)
X_train_std['RF']=X_train_std['RF'].apply(threshold)
X_test_std['ST']=X_test_std['ST'].apply(threshold)
X_test_std['RSF']=X_test_std['RSF'].apply(threshold)
X_test_std['FL']=X_test_std['FL'].apply(threshold)
X_test_std['PF']=X_test_std['PF'].apply(threshold)
X_test_std['RF']=X_test_std['RF'].apply(threshold)

In [25]:
from sklearn.svm import SVC
svm=SVC(kernel='linear', C=100, random_state=1, probability=True)

In [26]:
svm.fit(X_train_std, y_train)

SVC(C=100, kernel='linear', probability=True, random_state=1)

In [27]:
svm.score(X_train_std, y_train)

0.7171052631578947

In [28]:
svm.score(X_test_std, y_test)

0.696969696969697

In [29]:
from sklearn.ensemble import RandomForestClassifier

In [30]:
forest=RandomForestClassifier(max_depth=8, criterion='entropy', n_estimators=18, random_state=1)

In [31]:
forest.fit(X_train_std, y_train)

RandomForestClassifier(criterion='entropy', max_depth=8, n_estimators=18,
                       random_state=1)

In [32]:
forest.score(X_train_std, y_train)

0.9473684210526315

In [33]:
forest.score(X_test_std, y_test)

0.6060606060606061

In [85]:
from sklearn.ensemble import RandomForestClassifier
feat_labels=X.columns[0:]
forest=RandomForestClassifier(n_estimators=500, random_state=1)
forest.fit(X_train_std, y_train)
importances=forest.feature_importances_
indices=np.argsort(importances)[::-1]
for f in range(X_train_std.shape[1]):
    print("%2d) %-*s %f" % (f+1, 30, feat_labels[indices[f]], importances[indices[f]]))

 1) altitude                       0.115046
 2) disto_river                    0.101500
 3) disto_stations                 0.087607
 4) disto_mainroad                 0.087467
 5) disto_syorizyo                 0.084689
 6) inclination                    0.078180
 7) tilt_direction                 0.077671
 8) curvature                      0.070875
 9) no_water_days                  0.049586
10) up_down                        0.035410
11) popu-served                    0.023772
12) population_served              0.022420
13) pipelength                     0.019039
14) pipelength_per_pipe            0.017383
15) number_taps                    0.017268
16) (popu-served)/pipes            0.016934
17) oldest_pipe_age                0.016789
18) served/pipes                   0.015586
19) total_population               0.015464
20) source_ecoli                   0.014469
21) supply_hours                   0.013042
22) PF                             0.007060
23) ratio                       

In [83]:
X_train_std.columns

Index(['up_down', 'curvature', 'inclination', 'tilt_direction', 'altitude',
       'disto_river', 'disto_stations', 'disto_mainroad', 'disto_syorizyo',
       'supply_hours', 'no_water_days', 'total_population ',
       'population_served', 'popu-served', 'number_taps', 'pipelength',
       'pipelength_per_pipe', 'served/pipes', '(popu-served)/pipes',
       'oldest_pipe_age', 'ST', 'RSF', 'FL', 'PF', 'RF', 'ratio',
       'source_ecoli'],
      dtype='object')